In [1]:
import yaml
import requests
from argparse import ArgumentParser
import json
import pandas as pd
import random

In [2]:
path = "20230627-reserves-796140-20230627D100000108445000.yaml"
BASE_URL = 'https://blockstream.info/api/'

In [3]:
# Read YAML file
with open(path, 'r') as stream:
    data_loaded = yaml.safe_load(stream)


In [4]:
df = pd.DataFrame(data_loaded['address'])
print(df.shape)
df.head()

(315516, 4)


,addr_type,addr,script,balance
0,sh,3BMEXbSSrK2K7cRgqxrtqUWfxowBBrW1BE,534104220936c3245597b1513a9a7fe96d96facf1a840e...,171258
1,sh,3BMEX95VgAacEZRJksocYrPzJ328pcSFXG,534104220936c3245597b1513a9a7fe96d96facf1a840e...,830323
2,sh,3BMEXZXWZ2cZ7yS7UBkPNf1dRFRqPzV1xw,53410417db3fdd2f40aa63f68ad8db4bcb5a82037db40b...,1910899
3,sh,3BitMEXjLgKQHNMUsnmdbiVb462EP8jxi5,534104220936c3245597b1513a9a7fe96d96facf1a840e...,8622
4,sh,3BMEXf5prg81PF9Aj1eR5SLcrtZGw4kHah,534104220936c3245597b1513a9a7fe96d96facf1a840e...,300000


In [5]:
def get_address_info(address):
    # return info about address a
    url = BASE_URL + 'address/' + str(address)
    r = requests.get(url)
    d = json.loads(r.text)
    return d

def get_address_txs(address):
    txs = []
    # return list txs of all transactions for address
    # if a has more than 25 txs, we need to fetch all of them

    #################################

    # Implement this part

    url = BASE_URL + 'address/' + str(address) + '/txs/chain'
    r = requests.get(url)
    d = json.loads(r.text)
    lastLen = len(d)
    txs.extend(d)

    while lastLen == 25:
        url = BASE_URL + 'address/' + str(address) + '/txs/chain/' + d[24]["txid"]
        r = requests.get(url)
        d = json.loads(r.text)
        lastLen = len(d)
        txs.extend(d)

    #################################

    return txs

In [13]:
index_list = random.sample(range(df.shape[0]), 10)
print(index_list)

for index in index_list:
    addr = df["addr"].iloc[index]
    given_balance = df["balance"].iloc[index]

    addr_info = get_address_info(addr)
    addr_txs = get_address_txs(addr)

    results = dict()
    results['satoshi_in'] = addr_info["chain_stats"]["funded_txo_sum"]
    results['satoshi_out'] = addr_info["chain_stats"]["spent_txo_sum"]
    results['balance'] = addr_info["chain_stats"]["funded_txo_sum"] - addr_info["chain_stats"]["spent_txo_sum"]
    results['fees'] = sum(d['fee'] for d in addr_txs)
    results['b_in_first'] = 0
    results['b_out_first'] = 0
    results['n_neighbours_in'] = 0
    results['n_neighbours_out'] = 0
    
    '''
    txs = addr_txs

    if len(txs) > 0:

        recvs  = []
        sends = []

        first_recv = txs[0]["status"]["block_height"]
        first_sent = txs[0]["status"]["block_height"]

        for tx in txs:
            recvs_temp = []
            sends_temp = []

            for recv in tx["vin"]:
                if recv["prevout"]["value"] > 0 and "scriptpubkey_address" in recv["prevout"].keys():
                    recvs_temp.append(recv["prevout"]["scriptpubkey_address"])
            for send in tx["vout"]:
                if send["value"] > 0 and "scriptpubkey_address" in send.keys():
                    sends_temp.append(send["scriptpubkey_address"])

            # address has received when in sends and vice versa!
            if addr in recvs_temp:
                if first_sent > tx["status"]["block_height"]:
                    first_sent = tx["status"]["block_height"]
                sends.extend(sends_temp)
            if addr in sends_temp:
                if first_recv > tx["status"]["block_height"]:
                    first_recv = tx["status"]["block_height"]
                recvs.extend(recvs_temp)

        # again addresses that recv
        results['b_in_first'] = first_recv
        results['b_out_first'] = first_sent
        results['n_neighbours_in'] = len(list(set(recvs)))
        results['n_neighbours_out'] = len(list(set(sends)))
        
    #true_balance = results['b_in_first'] - results['b_out_first']
    '''
    
    true_balance = results['balance']
    
    #if(float(given_balance) != float(true_balance)):
    #    print(df.iloc[index])
    print("Compared "+str(given_balance)+" (given) with "+str(true_balance)+" (true)")
print("Done!")

[146102, 301095, 177735, 158084, 224561, 143248, 278748, 310124, 74945, 293446]
Compared 1640121 (given) with 1640121 (true)
Compared 2270174 (given) with 2270174 (true)
Compared 582635 (given) with 582635 (true)
Compared 2814433 (given) with 2814433 (true)
Compared 1450000 (given) with 1450000 (true)
Compared 437131 (given) with 437131 (true)
Compared 1012441753 (given) with 1012441753 (true)
Compared 30000 (given) with 30000 (true)
Compared 1100000 (given) with 1100000 (true)
Compared 2891963 (given) with 2891963 (true)
Done!


In [35]:
df["addr"].iloc[294544]

'bc1qmexyjccpv8vmujvxlcxgrwn8kcl9uptc8nsdshjgmwnhv5ny235sctxj34'